In [5]:
import pandas as pd
import numpy as np
import csv
import os

In [6]:
files = os.listdir('../Datasets/offensive_stats')
print(files)

['1997-1998.csv', '2011-2012.csv', '1992-1993.csv', '2008-2009.csv', '2010-2011.csv', '2020-2021.csv', '1993-1994.csv', '2006-2007.csv', '2021-2022.csv', '2019-2020.csv', '1994-1995.csv', '1995-1996.csv', '2016-2017.csv', '2000-2001.csv', '2018-2019.csv', '2001-2002.csv', '1999-2000.csv', '2004-2005.csv', '2022-2023.csv', '2005-2006.csv', '2013-2014.csv', '2023-2024.csv', '2012-2013.csv', '2017-2018.csv', '2007-2008.csv', '2002-2003.csv', '1998-1999.csv', '2015-2016.csv', '1996-1997.csv', '2003-2004.csv', '2014-2015.csv', '2009-2010.csv']


In [7]:
training_data = pd.read_csv('../Datasets/updated_training_data.csv')
original_data = pd.read_csv('../Datasets/premier-league-matches.csv')

training_data["Season"] = original_data['Season_End_Year'].apply(lambda x: f"{x-1}-{x}")


cols = ['Season'] + [i for i in training_data.columns.tolist() if i != 'Season']
training_data = training_data[cols]

training_data.to_csv('../Datasets/updated_training_data.csv', index=False)

In [8]:
season = 1992

# Use 'with' to open files for writing
with open('../Datasets/team_offensive_stats.csv', 'w', newline='') as offensive_file, \
     open('../Datasets/team_defensive_stats.csv', 'w', newline='') as defensive_file:

    offensive_writer = csv.writer(offensive_file)
    defensive_writer = csv.writer(defensive_file)

    offensive_writer.writerow(['Season', 'Team', 'Games', 'Goals', 'G+A', 'Penalties', 
                               'Avg Goals per Game', 'Avg G+A per Game', 'Avg Penalties per Game'])

    defensive_writer.writerow(['Season', 'Team', 'Games', 'Saves', 'Goals Conceded', 'Clean Sheets',
                               'Saves per game', 'Goals Conceded per game', 'Clean Sheets per game', 
                               'Saves%', 'Save Rate'])

    while season < 2023:
        try:
            # Read player data
            player_file = f'../Datasets/offensive_stats/{season}-{season+1}.csv'
            player_data = pd.read_csv(player_file)

            team_data = {}
            for index, row in player_data.iterrows():
                team = row['Squad']
                if team not in team_data:
                    team_data[team] = {"Goals": 0, "G+A": 0, "Penalties": 0}
                team_data[team]["Goals"] += row['Gls']
                team_data[team]["G+A"] += row['Gls'] + row['Ast']
                team_data[team]["Penalties"] += row['PK']

            for team in team_data:
                games_played = (
                    original_data[(original_data['Season_End_Year'] == season + 1) & (original_data['Home'] == team)].shape[0] +
                    original_data[(original_data['Season_End_Year'] == season + 1) & (original_data['Away'] == team)].shape[0]
                )

                offensive_writer.writerow([
                    f"{season}-{season + 1}", 
                    team, 
                    games_played, 
                    team_data[team]["Goals"], 
                    team_data[team]["G+A"], 
                    team_data[team]["Penalties"], 
                    team_data[team]["Goals"] / games_played if games_played > 0 else 0, 
                    team_data[team]["G+A"] / games_played if games_played > 0 else 0, 
                    team_data[team]["Penalties"] / games_played if games_played > 0 else 0
                ])

            # Read goalkeeper data
            keeper_file = f'../Datasets/keeping_stats/{season}-{season+1}.csv'
            keeper_data = pd.read_csv(keeper_file)

            team_keeping_data = {}
            for index, row in keeper_data.iterrows():
                team = row['Squad']
                if team not in team_keeping_data:
                    team_keeping_data[team] = {"Saves": 0, "Goals Conceded": 0, "Clean Sheets": 0, "Shots Against": 0}

                team_keeping_data[team]["Saves"] += row[' Saves']  # Remove space
                team_keeping_data[team]["Goals Conceded"] += row['Goals Against']
                team_keeping_data[team]["Clean Sheets"] += row['Clean Sheets']
                team_keeping_data[team]["Shots Against"] += row[' Shots on Target Against']  # Remove space

            # Write stats for each team
            for team in team_keeping_data:
                games_played = (
                    original_data[(original_data['Season_End_Year'] == season + 1) & (original_data['Home'] == team)].shape[0] +
                    original_data[(original_data['Season_End_Year'] == season + 1) & (original_data['Away'] == team)].shape[0]
                )

                saves_percentage = (
                    team_keeping_data[team]["Saves"] / 
                    (team_keeping_data[team]["Saves"] + team_keeping_data[team]["Goals Conceded"])
                    if (team_keeping_data[team]["Saves"] + team_keeping_data[team]["Goals Conceded"]) > 0 else 0
                )

                save_rate = (
                    team_keeping_data[team]["Saves"] / team_keeping_data[team]["Shots Against"]
                    if team_keeping_data[team]["Shots Against"] > 0 else 0
                )

                defensive_writer.writerow([
                    f"{season}-{season + 1}", 
                    team, 
                    games_played, 
                    team_keeping_data[team]["Saves"], 
                    team_keeping_data[team]["Goals Conceded"], 
                    team_keeping_data[team]["Clean Sheets"], 
                    team_keeping_data[team]["Saves"] / games_played if games_played > 0 else 0, 
                    team_keeping_data[team]["Goals Conceded"] / games_played if games_played > 0 else 0, 
                    team_keeping_data[team]["Clean Sheets"] / games_played if games_played > 0 else 0, 
                    saves_percentage, 
                    save_rate
                ])

        except Exception as e:
            print(f"Season {season}-{season+1} failed: {e}")

        print(f"Season {season}-{season+1} complete")
        season += 1


Season 1992-1993 complete
Season 1993-1994 complete
Season 1994-1995 complete
Season 1995-1996 complete
Season 1996-1997 complete
Season 1997-1998 complete
Season 1998-1999 complete
Season 1999-2000 complete
Season 2000-2001 complete
Season 2001-2002 failed: Error tokenizing data. C error: Expected 13 fields in line 20, saw 14

Season 2001-2002 complete
Season 2002-2003 complete
Season 2003-2004 complete
Season 2004-2005 complete
Season 2005-2006 complete
Season 2006-2007 failed: Error tokenizing data. C error: Expected 13 fields in line 25, saw 14

Season 2006-2007 complete
Season 2007-2008 complete
Season 2008-2009 complete
Season 2009-2010 complete
Season 2010-2011 complete
Season 2011-2012 complete
Season 2012-2013 complete
Season 2013-2014 complete
Season 2014-2015 complete
Season 2015-2016 complete
Season 2016-2017 complete
Season 2017-2018 complete
Season 2018-2019 complete
Season 2019-2020 complete
Season 2020-2021 complete
Season 2021-2022 complete
Season 2022-2023 complete
